In [1]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
import os
import sys
from dotenv import load_dotenv
from typing import List
from llama_index.core import Settings
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import BaseNode, TransformComponent
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.legacy.retrievers.bm25_retriever import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
import faiss

In [2]:
#Settings.llm = Ollama(model="llama3.1:latest", request_timeout=120.0)
from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings
Settings.llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
Settings.embedding = OllamaEmbedding(model_name="nomic-embed-text:v1.5")

In [3]:
EMBED_DIMENSION=512


In [4]:
# Create FaisVectorStore to store embeddings
fais_index = faiss.IndexFlatL2(EMBED_DIMENSION)
vector_store = FaissVectorStore(faiss_index=fais_index)

In [5]:
class TextCleaner(TransformComponent):
    """
    Transformation to be used within the ingestion pipeline.
    Cleans clutters from texts.
    """
    def __call__(self, nodes, **kwargs) -> List[BaseNode]:
        
        for node in nodes:
            node.text = node.text.replace('\t', ' ') # Replace tabs with spaces
            node.text = node.text.replace(' \n', ' ') # Replace paragprah seperator with spacaes
            
        return nodes

In [6]:
# Pipeline instantiation with: 
# node parser, custom transformer, vector store and documents
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        TextCleaner()
    ],
    vector_store=vector_store,
    )

# Run the pipeline to get nodes
lyft_docs = SimpleDirectoryReader(
        input_files=["../data/10k/lyft_2021.pdf"]
    ).load_data()
uber_docs = SimpleDirectoryReader(
        input_files=["../data/10k/uber_2021.pdf"]
    ).load_data()


In [7]:
nodes_lyft = pipeline.run(documents=lyft_docs)
nodes_uber = pipeline.run(documents=uber_docs)

In [8]:
lyft_index = VectorStoreIndex(nodes= nodes_lyft, embed_model=Settings.embedding)
lyft_index.storage_context.persist(persist_dir="../storage/lyft")

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
uber_index = VectorStoreIndex(nodes= nodes_uber, embed_model=Settings.embedding)
uber_index.storage_context.persist(persist_dir="../storage/uber")

In [10]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [11]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool. "
                "The input is used to power a semantic search engine."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool. "
                "The input is used to power a semantic search engine."
            ),
        ),
    ),
]

In [12]:

# needs llama_index-packs-agents-coa
from llama_index.packs.agents_coa import CoAAgentPack

pack = CoAAgentPack(tools=query_engine_tools, llm=Settings.llm)

In [13]:
import nest_asyncio
nest_asyncio.apply()
response = pack.run("How did Ubers revenue growth compare to Lyfts in 2021?")

==== Available Parsed Functions ====
def lyft_10k(input: string):
   """Provides information about Lyft financials for year 2021. Use a detailed plain text question as input to the tool. The input is used to power a semantic search engine."""
    ...
def uber_10k(input: string):
   """Provides information about Uber financials for year 2021. Use a detailed plain text question as input to the tool. The input is used to power a semantic search engine."""
    ...


/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/lib/python3.10/site-packages/llama_index/llms/langchain/base.py:95: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  lc_message = self._llm.predict_messages(messages=lc_messages, **kwargs)


==== Generated Chain of Abstraction ====
Abstract plan of reasoning:
To answer the question, we need to compare the revenue growth of Uber and Lyft in 2021. We will first get the revenue information for Lyft in 2021 by asking [FUNC lyft_10k("What was Lyft's revenue in 2021 and how did it change from 2020 to 2021?") = y1]. Then, we will get the revenue information for Uber in 2021 by asking [FUNC uber_10k("What was Uber's revenue in 2021 and how did it change from 2020 to 2021?") = y2]. Finally, we will compare the revenue growth of Uber and Lyft in 2021 by calculating the difference between y2 and y1.
==== Executing lyft_10k with inputs ["What was Lyft's revenue in 2021 and how did it change from 2020 to 2021?"] ====
==== Executing uber_10k with inputs ["What was Uber's revenue in 2021 and how did it change from 2020 to 2021?"] ====


In [14]:
print(str(response))


Based on the previous reasoning, we can conclude that Lyft's revenue growth in 2021 was 36% compared to 2020. However, due to the lack of explicit information about Uber's total revenue in 2021, we cannot accurately compare the revenue growth of Uber and Lyft in 2021. We can only infer that Uber's revenue in 2021 was lower than in 2020, with a significant decrease in All Other revenue, but the exact amount of the decrease is not specified.
